<a href="https://colab.research.google.com/github/betoloayma-ship-it/challenge1-data-science-latam/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [5]:
import requests
import pandas as pd

# URL "Raw" de tu archivo JSON en GitHub
url_api = "https://raw.githubusercontent.com/betoloayma-ship-it/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"

try:
    # 1. Extracción de datos desde la URL
    respuesta = requests.get(url_api)
    respuesta.raise_for_status() # Verifica si la solicitud fue exitosa (código 200)

    # 2. Carga de los datos JSON en un DataFrame de pandas
    datos = pd.json_normalize(respuesta.json())

    print("✅ ¡Datos extraídos exitosamente desde tu GitHub!")
    print("\n--- Explorando las Columnas y Tipos de Datos ---")

    # 3. Exploración inicial usando .info()
    datos.info()

except Exception as e:
    print(f"❌ Ocurrió un error al intentar extraer los datos: {e}")

✅ ¡Datos extraídos exitosamente desde tu GitHub!

--- Explorando las Columnas y Tipos de Datos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  inter

In [7]:
import pandas as pd
import numpy as np

# Asumimos que el DataFrame 'datos' ya fue cargado desde la API en el paso anterior.
# Si no, primero ejecuta el script para cargar los datos desde la URL de GitHub.
# url_api = "https://raw.githubusercontent.com/betoloayma-ship-it/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
# datos = pd.read_json(url_api)


# --- 1. Corregir Errores de Formato y Tipos de Datos ---
print("--- 1. Corrigiendo tipos de datos ---")

# a. Corregir la columna 'account.Charges.Total'
# Convertimos a número, los errores (espacios en blanco) se transformarán en NaN (Nulos)
datos['account.Charges.Total'] = pd.to_numeric(datos['account.Charges.Total'], errors='coerce')

# b. Estandarizar la columna 'Churn' a formato numérico (0 y 1)
# Esto facilitará los cálculos y el modelado más adelante
datos['Churn'] = datos['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

print("Tipos de datos después de la corrección:")
print(datos[['account.Charges.Total', 'Churn']].dtypes)


# --- 2. Verificación y Tratamiento de Valores Ausentes (Nulos) ---
print("\n--- 2. Verificando valores nulos ---")
print(datos.isnull().sum())

# Decidimos qué hacer con los nulos. Como son pocos (solo en 'account.Charges.Total'),
# una buena estrategia es eliminarlos.
datos.dropna(inplace=True)
print("\nSe eliminaron las filas con valores nulos.")
print("Verificación final de nulos (debería ser 0):")
print(datos.isnull().sum().sum())


# --- 3. Verificación de Datos Duplicados ---
print("\n--- 3. Verificando filas duplicadas ---")
duplicados = datos.duplicated().sum()
print(f"Número de filas duplicadas encontradas: {duplicados}")
# Si se encontraran duplicados, se eliminarían con: datos.drop_duplicates(inplace=True)


# --- 4. Análisis de Inconsistencias en Variables Categóricas ---
print("\n--- 4. Verificando consistencia en categorías ---")
# Seleccionamos solo las columnas de tipo 'object' (texto)
columnas_categoricas = datos.select_dtypes(include=['object']).columns

for columna in columnas_categoricas:
    print(f"\nCategorías únicas en '{columna}':")
    # Usamos .unique() para ver todas las categorías presentes
    print(datos[columna].unique())

print("\n\n✅ Proceso de limpieza y validación completado.")

--- 1. Corrigiendo tipos de datos ---
Tipos de datos después de la corrección:
account.Charges.Total    float64
Churn                      int64
dtype: object

--- 2. Verificando valores nulos ---
customerID                    0
Churn                         0
customer.gender               0
customer.SeniorCitizen        0
customer.Partner              0
customer.Dependents           0
customer.tenure               0
phone.PhoneService            0
phone.MultipleLines           0
internet.InternetService      0
internet.OnlineSecurity       0
internet.OnlineBackup         0
internet.DeviceProtection     0
internet.TechSupport          0
internet.StreamingTV          0
internet.StreamingMovies      0
account.Contract              0
account.PaperlessBilling      0
account.PaymentMethod         0
account.Charges.Monthly       0
account.Charges.Total        11
dtype: int64

Se eliminaron las filas con valores nulos.
Verificación final de nulos (debería ser 0):
0

--- 3. Verificando filas d

#🔧 Transformación

In [10]:
import pandas as pd
import numpy as np
import requests

# --- 1. Extracción de Datos (Paso previo) ---
# Se asume que este paso ya se ejecutó y tenemos el DataFrame 'datos'.
# Si no, se puede ejecutar de nuevo:
url_api = "https://raw.githubusercontent.com/betoloayma-ship-it/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
try:
    respuesta = requests.get(url_api)
    respuesta.raise_for_status() # Verifica si la solicitud fue exitosa (código 200)
    datos = pd.json_normalize(respuesta.json())
    print("✅ Datos extraídos correctamente.")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")
    # Si la carga falla, creamos un DataFrame vacío para evitar errores posteriores.
    datos = pd.DataFrame()

# --- 2. Limpieza y Transformación ---

if not datos.empty:
    # a. Corregir la columna 'account.Charges.Total'
    # Se convierte a número. Los errores (espacios en blanco) se transforman en NaN (Nulos).
    datos['account.Charges.Total'] = pd.to_numeric(datos['account.Charges.Total'], errors='coerce')

    # b. Eliminar filas con valores nulos
    # Como los nulos en 'account.Charges.Total' corresponden a clientes nuevos sin historial,
    # es seguro eliminarlos para el análisis de churn.
    datos.dropna(inplace=True)

    # c. Verificar y eliminar duplicados
    if datos.duplicated().sum() > 0:
        datos.drop_duplicates(inplace=True)
        print("Se eliminaron filas duplicadas.")

    # d. Estandarizar y transformar columnas categóricas a formato numérico (0 y 1)
    # Se convierte 'Yes'/'No' y 'Male'/'Female' a 1/0 para facilitar el análisis.
    columnas_a_transformar = [
        'customer.Partner', 'customer.Dependents', 'phone.PhoneService', 'account.PaperlessBilling', 'Churn'
    ]
    for columna in columnas_a_transformar:
        # Se asegura de que la columna exista antes de transformarla
        if columna in datos.columns:
            datos[columna] = datos[columna].apply(lambda x: 1 if x == 'Yes' else 0)

    # Transformar la columna 'customer.gender'
    if 'customer.gender' in datos.columns:
        datos['customer.gender'] = datos['customer.gender'].apply(lambda x: 1 if x == 'Male' else 0)

    # e. Eliminar la columna 'customerID' que no es útil para el modelado
    if 'customerID' in datos.columns:
        datos = datos.drop('customerID', axis=1)

    print("\n--- Verificación Final del DataFrame Limpio ---")
    datos.info()

    print("\n--- Primeras 5 filas de los datos transformados ---")
    print(datos.head())

✅ Datos extraídos correctamente.

--- Verificación Final del DataFrame Limpio ---
<class 'pandas.core.frame.DataFrame'>
Index: 7256 entries, 0 to 7266
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Churn                      7256 non-null   int64  
 1   customer.gender            7256 non-null   int64  
 2   customer.SeniorCitizen     7256 non-null   int64  
 3   customer.Partner           7256 non-null   int64  
 4   customer.Dependents        7256 non-null   int64  
 5   customer.tenure            7256 non-null   int64  
 6   phone.PhoneService         7256 non-null   int64  
 7   phone.MultipleLines        7256 non-null   object 
 8   internet.InternetService   7256 non-null   object 
 9   internet.OnlineSecurity    7256 non-null   object 
 10  internet.OnlineBackup      7256 non-null   object 
 11  internet.DeviceProtection  7256 non-null   object 
 12  internet.TechSupport       

In [12]:
import pandas as pd


# --- Creación de la nueva columna 'Cuentas_Diarias' ---

# Usamos 30.44 como el promedio de días en un mes (365.25 / 12) para mayor precisión.
promedio_dias_mes = 30.44
datos['Cuentas_Diarias'] = datos['account.Charges.Monthly'] / promedio_dias_mes

# --- Verificación del Resultado ---
# Mostramos la columna original y la nueva para comparar
print("✅ Columna 'Cuentas_Diarias' creada exitosamente.")
print("Mostrando las primeras 5 filas con los cargos mensuales y diarios:")
print(datos[['account.Charges.Monthly', 'Cuentas_Diarias']].head())

✅ Columna 'Cuentas_Diarias' creada exitosamente.
Mostrando las primeras 5 filas con los cargos mensuales y diarios:
   account.Charges.Monthly  Cuentas_Diarias
0                     65.6         2.155059
1                     59.9         1.967806
2                     73.9         2.427727
3                     98.0         3.219448
4                     83.9         2.756242


In [15]:
import pandas as pd
import numpy as np
import requests

# --- 1. Carga de Datos ---
url_api = "https://raw.githubusercontent.com/betoloayma-ship-it/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
try:
    respuesta = requests.get(url_api)
    respuesta.raise_for_status() # Verifica si la solicitud fue exitosa (código 200)
    datos = pd.json_normalize(respuesta.json())
    print("✅ Datos cargados correctamente.")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")
    datos = pd.DataFrame()


if not datos.empty:
    # --- 2. Diagnóstico del Problema ---
    # Verificamos el tipo de dato de 'account.Charges.Total' ANTES de cualquier cambio.
    # El resultado probablemente será 'object', que significa texto.
    print(f"\nTipo de dato original de 'account.Charges.Total': {datos['account.Charges.Total'].dtype}")


    # --- 3. Aplicación de la Solución ---
    # a. Convertir la columna a tipo numérico.
    #    errors='coerce' es la clave: transforma cualquier valor que no sea un número en NaN (Nulo).
    datos['account.Charges.Total'] = pd.to_numeric(datos['account.Charges.Total'], errors='coerce')

    # b. Manejar los valores nulos (NaN) que se crearon en el paso anterior.
    #    La forma más sencilla y segura es eliminar las filas completas.
    datos.dropna(inplace=True)
    print("\nSe convirtieron los 'account.Charges.Total' a número y se eliminaron las filas con errores.")


    # --- 4. Verificación Final ---
    # Verificamos que el tipo de dato ahora sea numérico (como float64)
    # y que ya no haya valores nulos en esa columna.
    print(f"Nuevo tipo de dato de 'account.Charges.Total': {datos['account.Charges.Total'].dtype}")
    print(f"Valores nulos restantes en 'account.Charges.Total': {datos['account.Charges.Total'].isnull().sum()}")


    print("\n✅ ¡Problema solucionado! El DataFrame está listo para continuar.")

✅ Datos cargados correctamente.

Tipo de dato original de 'account.Charges.Total': object

Se convirtieron los 'account.Charges.Total' a número y se eliminaron las filas con errores.
Nuevo tipo de dato de 'account.Charges.Total': float64
Valores nulos restantes en 'account.Charges.Total': 0

✅ ¡Problema solucionado! El DataFrame está listo para continuar.


#📊 Carga y análisis

In [19]:
# --- Generación del Análisis Descriptivo ---

# .describe() calcula las estadísticas para todas las columnas numéricas.
# .round(2) redondea los resultados a 2 decimales para mayor claridad.
analisis_descriptivo = datos.describe().round(2)

print("--- Análisis Descriptivo de las Variables Numéricas ---")
print(analisis_descriptivo)

--- Análisis Descriptivo de las Variables Numéricas ---
       customer.SeniorCitizen  customer.tenure  account.Charges.Monthly  \
count                 7256.00          7256.00                  7256.00   
mean                     0.16            32.40                    64.76   
std                      0.37            24.56                    30.13   
min                      0.00             1.00                    18.25   
25%                      0.00             9.00                    35.45   
50%                      0.00            29.00                    70.30   
75%                      0.00            55.00                    89.90   
max                      1.00            72.00                   118.75   

       account.Charges.Total  
count                7256.00  
mean                 2280.63  
std                  2268.63  
min                    18.80  
25%                   400.22  
50%                  1391.00  
75%                  3785.30  
max                 

In [27]:
import requests
import pandas as pd

# La misma URL de tu GitHub
url_api = "https://raw.githubusercontent.com/betoloayma-ship-it/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"

try:
    # Paso 1: Obtener los datos de la API
    respuesta = requests.get(url_api)
    respuesta.raise_for_status() # Verificar si la solicitud fue exitosa
    datos_json = respuesta.json()

    # Paso 2: Usar json_normalize para aplanar la estructura de los datos
    datos = pd.json_normalize(datos_json)

    print("✅ ¡Datos anidados extraídos y aplanados exitosamente!")

    # Paso 3: Inspeccionar las columnas AHORA SÍ aplanadas
    print("\nNombres de columna después de aplanar el JSON:")
    print(datos.columns.tolist())

    # Paso 4: Mostrar las primeras filas para verificar la tabla correcta
    print("\nPrimeras 5 filas del DataFrame correcto:")
    print(datos.head())

except Exception as e:
    print(f"❌ Ocurrió un error: {e}")

✅ ¡Datos anidados extraídos y aplanados exitosamente!

Nombres de columna después de aplanar el JSON:
['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen', 'customer.Partner', 'customer.Dependents', 'customer.tenure', 'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService', 'internet.OnlineSecurity', 'internet.OnlineBackup', 'internet.DeviceProtection', 'internet.TechSupport', 'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract', 'account.PaperlessBilling', 'account.PaymentMethod', 'account.Charges.Monthly', 'account.Charges.Total']

Primeras 5 filas del DataFrame correcto:
   customerID Churn customer.gender  customer.SeniorCitizen customer.Partner  \
0  0002-ORFBO    No          Female                       0              Yes   
1  0003-MKNFE    No            Male                       0               No   
2  0004-TLHLJ   Yes            Male                       0               No   
3  0011-IGKFF   Yes            Male               

In [28]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. Carga y Limpieza (Pasos previos) ---
url_api = "https://raw.githubusercontent.com/betoloayma-ship-it/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
try:
    datos = pd.read_json(url_api)
    # Aplicamos las transformaciones clave
    datos['TotalCharges'] = pd.to_numeric(datos['TotalCharges'], errors='coerce')
    datos.dropna(inplace=True)
    if 'customerID' in datos.columns:
        datos = datos.drop('customerID', axis=1)
    if 'Churn' in datos.columns:
        datos.rename(columns={'Churn': 'Evasion'}, inplace=True)
        datos['Evasion'] = datos['Evasion'].apply(lambda x: 1 if x == 'Yes' else 0)
    print("✅ Datos cargados y preparados.")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")
    datos = pd.DataFrame() # DataFrame vacío si falla


# --- Diagnóstico ---
# Verificamos si el DataFrame está vacío antes de intentar graficar
print(f"\n¿El DataFrame está vacío?: {datos.empty}")


# --- 2. Visualización de la Distribución de Churn ---
if not datos.empty:
    print("El DataFrame contiene datos. Procediendo a crear los gráficos...")

    # Crear una figura con dos subplots
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Gráfico de Barras (Count Plot)
    sns.countplot(ax=axes[0], x='Evasion', data=datos)
    axes[0].set_title('Distribución de Clientes (Cantidad)')
    axes[0].set_xticklabels(['No Cancela (0)', 'Sí Cancela (1)'])
    axes[0].set_xlabel('Estado del Cliente')
    axes[0].set_ylabel('Número de Clientes')

    # Gráfico Circular (Pie Chart)
    churn_counts = datos['Evasion'].value_counts()
    axes[1].pie(churn_counts, labels=['No Cancela', 'Sí Cancela'], autopct='%1.1f%%', startangle=90, colors=['#66b3ff','#ff9999'])
    axes[1].set_title('Proporción de Clientes por Evasión (Churn)')

    plt.tight_layout()

    # Comando para mostrar el gráfico
    plt.show()
else:
    print("El DataFrame está vacío, por lo tanto, no se pueden generar los gráficos.")

❌ Error al cargar datos: 'TotalCharges'

¿El DataFrame está vacío?: True
El DataFrame está vacío, por lo tanto, no se pueden generar los gráficos.


#📄Informe final